# RAG 应用评估

RAG 应用评估是一个复杂的问题，完整的评估有很多指标。

## 1. 相关开源项目

### 1.1 [ragas](https://github.com/explodinggradients/ragas)

开源框架 ragas 有如下指标。

| 阶段 | 指标名称 | 指标描述 | 评估方法 | 是否需要参考答案(`ground truth`) |
| --- | --- | --- | --- | --- | 
| 检索 | 上下文召回 Context Recall | 检索到的上下文`context`与参考答案`ground truth`的一致性程度 | LLM | 是 |
| 检索 | 上下文精确度 Context Precision | 评估检索到的上下文`context`中，和参考答案`ground truth`一致的结果是否靠前 | LLM | 是 |
| 检索 | 上下文相关性 Context Relevancy | 评估检索到的上下文`context`中，和问题`question`的相关程度 | LLM | 否 |
| 生成 | 回答相关性 Answer Relevancy | 问题`question` 和答案 `answer`的相关性 | LLM | 否 |
| 生成 | 回答语义相似性 Answer semantic similarity | 答案 `answer`和参考答案`ground truth`的语义相似度 | 交叉编码器 | 是 |
| 生成 | 回答正确性 Answer Correctness | 答案 `answer`和参考答案`ground truth` 在事实方面的一致性 | LLM | 是 |
| 生成 | 回答忠诚度 Answer Faithfulness | 答案 `answer` 和上下文 `context`的事实一致性 | LLM | 否 |
| 生成 | 回答批评 Answer Critique | 对答案 `answer` 在指定的 Prompt 进行批评以识别有害内容 | LLM | 否 |

当数据集中没有人工标注的参考答案时，就需要使用那些不需要参考答案的指标，例如上下文相关性、回答相关性、回答忠诚度、回答批评等。

如果觉得评估指标过多，在有参考答案时，建议选择如下两个指标：

1. 上下文召回：用来评估检索效果。注意这里如果效果下降，可能原因是检索本身的效果不好，也可能是数据源缺少相关的数据。
2. 回答正确性：用来评估生成效果。

注：开源框架 ragas 中的 Prompt 和交叉编码器都是英文，不适合中文环境。

### 1.2 [tvalmetrics](https://github.com/TonicAI/tvalmetrics)

| Metric Name              | Inputs                                                    | Formula | What does it measure? | Which components does it evaluate? |
| ----------------------- | --------------------------------------------------------- | ------------------------------------------------------------ | ------------------------------------ |----|
| **Answer similarity score** | Question + Reference answer + LLM answer | Score between 0 and 5 | How well the reference answer matches the LLM answer. | All components.                     |
| **Retrieval precision** | Question + Retrieved context                         | (Count of relevant retrieved context) / (Count of retrieved context) | Whether the context retrieved is relevant to answer the given question. | Chunker + Embedder + Retriever    |
| **Augmentation precision** | Question + Retrieved context + LLM answer             | (Count of relevant retrieved context in LLM answer) / (Count of relevant retrieved context) | Whether the relevant context is in the LLM answer. | Prompt builder + LLM                |
| **Augmentation accuracy** | Retrieved context + LLM answer                          | (Count of retrieved context in LLM answer) / (Count of retrieved context) | Whether all the context is in the LLM answer. | Prompt builder + LLM                |
| **Answer consistency** or **Answer consistency binary** | Retrieved context + LLM answer                          | (Count of the main points in the answer that can be attributed to context) / (Count of main points in answer) | Whether there is information in the LLM answer that does not come from the context. | Prompt builder + LLM                |
| **Retrieval k-recall** | Question + Retrieved context + Top k context         | (Count of relevant retrieved context) / (Count of relevant context in top k context) | How well the retrieval system retrieves all of the relevant context. | Chunker + Embedder + Retriever    |




## 2. 数据集准备

评测 RAG 应用，数据集必须有：

- 运行输入：
    - question[str]：问题
- 运行输出
    - answer[str]：RAG 应用给出的回答
    - contexts[list[str]]: 检索到的上下文，顺序则代表相似度。
- 评估输入
    - reference_context[str]: 参考上下文，用于评估检索的正确性。
    - reference_answer[str]: 参考答案，用于评估回答的正确性。

数据集有三种方法准备：

1. 使用开源数据集，比如 ragas 引用的 explodinggradients/fiqa 数据集。但是中文的 RAG 数据集较少。而且开源数据集只能代表 RAG 的通用能力，不能代表 RAG 在特定领域的能力。
2. 人工标注，这种方法需要大量的人力成本，但是可以标注特定领域的数据集。
3. 使用 LLM 自动抽取 QA 对，从而形成数据集。这种方法的优点是成本低，缺点是数据集的质量可能不高。自动标注尽量使用能力较强的 LLM，比如 GPT-4 等。

### 2.1 使用开源数据集

从项目 [m3e-base](https://huggingface.co/moka-ai/m3e-base) 介绍中，可以看到作者收集的众多用来训练 Embedding 模型的中文数据集，从其中挑选问答类数据集处理后可用于 RAG 评估。

筛选后，[dureader_robust](https://huggingface.co/datasets/PaddlePaddle/dureader_robust/viewer/plain_text/train?row=96) 和 [cmrc2018](https://huggingface.co/datasets/cmrc2018) 比较适合用于 RAG 评估。前者的回答过于简略，所以我们选择 cmrc2018 数据集进行评估。

注意：cmrc2018 数据集并没有允许商用，请不要在商业项目中使用。

#### 2.1.1 cmrc 数据集下载和转换

In [ ]:
%pip install datasets

In [11]:
from datasets import load_dataset

cmrc = load_dataset("cmrc2018")
print(cmrc["test"][0])
cmrc

{'id': 'TRIAL_800_QUERY_0', 'context': '基于《跑跑卡丁车》与《泡泡堂》上所开发的游戏，由韩国Nexon开发与发行。中国大陆由盛大游戏运营，这是Nexon时隔6年再次授予盛大网络其游戏运营权。台湾由游戏橘子运营。玩家以水枪、小枪、锤子或是水炸弹泡封敌人(玩家或NPC)，即为一泡封，将水泡击破为一踢爆。若水泡未在时间内踢爆，则会从水泡中释放或被队友救援(即为一救援)。每次泡封会减少生命数，生命数耗完即算为踢爆。重生者在一定时间内为无敌状态，以踢爆数计分较多者获胜，规则因模式而有差异。以2V2、4V4随机配对的方式，玩家可依胜场数爬牌位(依序为原石、铜牌、银牌、金牌、白金、钻石、大师) ，可选择经典、热血、狙击等模式进行游戏。若游戏中离，则4分钟内不得进行配对(每次中离+4分钟)。开放时间为暑假或寒假期间内不定期开放，8人经典模式随机配对，采计分方式，活动时间内分数越多，终了时可依该名次获得奖励。', 'question': '生命数耗完即算为什么？', 'answers': {'text': ['踢爆'], 'answer_start': [127]}}


DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

正式项目中，应该以全部的数据作为我们的向量索引的基础数据，单条数据格式如上。我们挑选其中的 context 嵌入到向量索引中。

本次我们就只以 test 的 1000 条数据建立索引。同时以 test 的随机 50 条查询作为验证数据集。

此时我们可以准备两种数据集

1. 用于运行的数据集，即只有 question / reference_answer / reference_contexts 的数据集。
2. 用于评估的数据集，即使用 chain 检索后的数据集，包含全部数据。

前者在启动评估任务的时候还要注册 Provider 用于 RAG 生成，为模拟真实的情况，我们选择 1。那么此处只需要生成用于运行的数据集即可。


In [73]:
test_data = []
for i in cmrc["test"].shuffle(seed=42).select(range(100)):
    test_data.append({
        "question": i["question"],
        "reference_context": i["context"],
        "reference_answer": i["answers"]["text"][0],
    })
import pandas as pd
pd.DataFrame(test_data).head()

,question,reference_context,reference_answer
0,与藤原纪香的婚礼耗资多少日圆？,阵内智则（1974年2月22日－），日本喜剧演员及主持。曾经于日本节目《娱乐之神》表演多场短...,5亿日圆
1,此剧的制作人是谁？,《西南忠魂》是台湾台湾电视公司于1984年（民国73年）3月28日至1985年（民国74年）...,制作人伍宗德
2,达曼的卫星城有哪几座？,达曼（）位于沙特阿拉伯的东部省，是沙特石油工业的重要中心。达曼是东部省最大的城市，达曼港也是...,达曼的卫星城有现代经济中心 Khobar、世界最大的沙特Aramco石油公司所在地札哈兰以及...
3,《This Is Where I Came In》是Bee Gees的第几张原创专辑？,《This Is Where I Came In》是Bee Gees的第20张原创专辑，也是...,第20张
4,卡洛身兼几家骑士团的大团长职位？,卡洛·玛利亚·贝尔纳多·真纳罗（Carlo Maria Bernardo Gennaro，）...,圣乔治康斯坦丁骑士团、圣斐迪南骑士团、圣真纳罗骑士团和弗朗切斯科一世王家骑士团


In [74]:
import json
with open("cmrc-eval-zh.jsonl", "w") as f:
    for i in test_data:
        f.write(json.dumps(i, ensure_ascii=False) + "\n")

#### 2.1.2 生成 FAISS 向量检索嵌入

Embedding 模型我们使用目前 cmteb reranking 任务 SOTA 的模型：[stella-base-zh](https://huggingface.co/infgrad/stella-base-zh)

In [17]:
!pip install langchain openai faiss-cpu tiktoken sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 218.4 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 584.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.4/165.4 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.8 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 9.9 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 8.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 9.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 12.4 MB/s eta 0:

In [22]:
# 正式项目中，应该以全部的数据作为我们的向量索引的基础数据，单条数据格式如上。我们挑选其中的 context 嵌入到向量索引中。
# 本次我们就只以 test 的 1000 条数据建立索引。
# Embedding 模型我们使用目前 cmteb reranking 任务 SOTA 的模型：[stella-base-zh](https://huggingface.co/infgrad/stella-base-zh)

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS

# 从 cmrc 中提取 context
texts_set = set()
texts = []
metadatas = []
for i in cmrc["test"]:
    if i["context"] not in texts_set:
        # 去重
        texts.append(i["context"])
        metadatas.append({"id": i["id"]})
        texts_set.add(i["context"])
print(texts[0])
print(metadatas[0])
print(len(texts))

基于《跑跑卡丁车》与《泡泡堂》上所开发的游戏，由韩国Nexon开发与发行。中国大陆由盛大游戏运营，这是Nexon时隔6年再次授予盛大网络其游戏运营权。台湾由游戏橘子运营。玩家以水枪、小枪、锤子或是水炸弹泡封敌人(玩家或NPC)，即为一泡封，将水泡击破为一踢爆。若水泡未在时间内踢爆，则会从水泡中释放或被队友救援(即为一救援)。每次泡封会减少生命数，生命数耗完即算为踢爆。重生者在一定时间内为无敌状态，以踢爆数计分较多者获胜，规则因模式而有差异。以2V2、4V4随机配对的方式，玩家可依胜场数爬牌位(依序为原石、铜牌、银牌、金牌、白金、钻石、大师) ，可选择经典、热血、狙击等模式进行游戏。若游戏中离，则4分钟内不得进行配对(每次中离+4分钟)。开放时间为暑假或寒假期间内不定期开放，8人经典模式随机配对，采计分方式，活动时间内分数越多，终了时可依该名次获得奖励。
{'id': 'TRIAL_800_QUERY_0'}
256


In [ ]:
# init embedding
embed = HuggingFaceEmbeddings(model_name="infgrad/stella-base-zh")
embed.embed_query("你好")

In [25]:
# load to vectorstore
vectorstore = FAISS.from_texts(
    texts=texts,
    embedding=embed,
    metadatas=metadatas,
)
vectorstore.similarity_search("你好")

[Document(page_content='牛佬，香港漫画家，原名文启明，1961年10月15日出生，生肖属牛，已婚无子女，少年时成长于蓝田廉租屋，中学二年级时辍学。12岁开始从事漫画行业，其时七十年代漫画行业百花齐放，「玉郎图书公司」、「小宝出版社」及「保光出版社」三分漫画天下，同期亦有漫画报纸《光报》、《喜报》、《青报》、《金报》、《生报》等，文氏也先后曾辗转加入过。13岁跟随漫画师父上官玉郎（莫君岳）学习，14岁成为上官小宝（邝东源）入室弟子，黄钧岳亦是牛佬入行的启蒙老师，到了八十年代初再加入玉郎机构，曾编绘《金刚》及为《如来神掌》起稿。1986年加入「邝氏」（八二画社），制作《爱情故事》，任《鬼书皇II我若为皇》编剧及监制，主编《江湖大佬》至1992年离开。在1992年4月1日与陈科琳、文鉴鸿、邱瑞新及伦裕国成立「浩一有限公司」，开始了其代表作《古惑仔》，之后自己成立现今的「和平出版有限公司」，其同父异母兄长文国兆Jacky亦帮手在「和平出版有限公司」处理行政工作。兴趣是烟不离手、游泳、养狗、潜水和写作，曾在年青时学习过中国武术，西洋拳、空手道、自由搏击及跆拳道，近年多在公司内操练泰拳，牛佬先后考获潜水Diver Master潜水长资格，及一级（基础）泰拳教练资格。歌手偶像是罗大佑、麦当娜及张学友，喜爱的电影导演则有徐克、黑泽明及史提芬·史匹堡，欣赏的漫画家则有永安巧及大友克洋。幼年时代邻居觉得其声线沉闷如牛，鼻大似牛，个性像牛，于是得「牛佬」称号。牛佬重视培育漫画人材，曾成立「牛家班」。伦裕国、邱瑞新、温日良、毕亦乐、胡达泉、颜子健及吴文辉都曾受到牛佬的指导。牛佬是香港漫画家中最多旗下漫画角色被拍成电影；亦是第一个旗下漫画被用作为电脑上线游戏的蓝本;更加是第一个推出三日刊的香港漫画家。', metadata={'id': 'TRIAL_920_QUERY_0'}),
 Document(page_content='《捉鬼男》（）美国一部由Tara Butters和Michele Fazekas创造的电视剧，首集是由Kevin Smith执导。Sam Oliver (Bret Harrison饰)Bert "Sock" Wysocki (饰)Benjamin "Benji" Gonzalez (饰)Andi Prendergast (饰)Josie Mi

In [50]:
vectorstore.save_local("cmrc-eval-zh.faiss")

#### 2.1.3 简单测试一下 RAG 检索效果

In [27]:
import getpass
openai_api_key = getpass.getpass("Please input your openai api key: ")

In [61]:
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

retriever = vectorstore.as_retriever()

template = """仅使用如下上下文回答问题：
```
{context}
```

问题：{question}
回答：
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(openai_api_key=openai_api_key)

def _combine_documents(docs):
    return "\n\n".join([i.page_content for i in docs])

_inputs = RunnablePassthrough()

retrieved_documents = {
    "docs": itemgetter("question") | retriever,
    "question": itemgetter("question"),
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | prompt | model,
    "docs": itemgetter("docs"),
}

final_chain = _inputs | retrieved_documents | answer


In [63]:
index = 10
q = cmrc["test"][index]["question"]
print(q)
print(final_chain.invoke({"question": q}))
print(cmrc["test"][index]["answers"]["text"][0])

松平康忠什么时候被任命为知行？
[Document(page_content='松平康忠（1546年－1618年9月28日）是日本战国时代至安土桃山时代的武将。长泽松平家第8代当主。德川氏的家臣。德川家康的从弟。父亲是松平政忠。母亲是松平清康的女儿碓井姬。正室是矢田姬（松平广忠的女儿，家康的妹妹）。继室是松平信定的女儿。通称源七郎、上野介。在天文15年（1546年）出生。父亲政忠在永禄3年（1560年）5月的桶狭间之战中被讨死，祖父亲广成为年幼的康忠的后见。另一方面，母亲碓井姬再嫁给德川家的重臣酒井忠次，于是与康忠分开。在永禄5年 （1562年）元服，被任命为三河国宝饭郡小坂井等1千8百10贯文的知行。康忠的叔父信重和近清等人亦被家康给予1百贯文并任命为辅佐。信重在翌年的三河一向一揆属于家康方被讨死，而近清一直辅助康忠直至天正16年（1588年）死去。此后康忠在元龟元年（1570年）从属于义父忠次参加姉川之战。接著在天正3年（1575年）的长筿之战中并跟随忠次参战。作为德川军的别働队攻略武田信实守备的鸢巢砦，成为帮助长筿城解围。之后成为家康的嫡男信康的家老，不过因为信康在天正7年（1579年）自杀而蛰居。后来得到家康的允许而复归，在天正10年（1582年）的本能寺之变发生后与家康一同穿越伊贺。之后参加天正12年（1584年）的小牧长久手之战。在天正16年（1588年）把家督让予嫡子康直并在京都隐居。不久，成为武藏深谷藩藩主的康直在文禄2年（1593年）以24歳之龄病死。已经隐居的康忠收家康的七男松千代为康直的养子并令其继承深谷藩1万石。不过松千代亦在庆长4年（1599年）死去，因此以松千代的哥哥辰千代（后来的松平忠辉）为继承人。在元和4年（1618年）8月10日死去。享年73岁。法号是源斋。在元和2年（1616年）因为忠辉被改易而令长泽松平家嫡流绝后，不过康忠的血脉亦有存续到后世。在天文9年（1540年）的安城合战中战死的同名武将松平康忠（甚六郎）是松平宗家亲忠系的松平张忠的儿子，不是同一人。', metadata={'id': 'TRIAL_598_QUERY_0'}), Document(page_content='折上原之战（），在日本战国时代的1589年7月17日发生，是伊达政宗对芦名义广和佐竹义重的一场战役。人取桥之战后，伊达政宗与芦名氏和佐竹氏对立表面化

### 2.2 使用 LLM 标注数据集

TODO，参考 https://docs.llamaindex.ai/en/stable/examples/evaluation/QuestionGeneration.html

主要方法是先拆分 context，然后对每段 context 使用 LLM 抽取 QA 对，从而形成数据集。

## 3. Embedding 模型评估和微调

可以使用 c-mteb 中 Reranking 方法评估检索模型的效果。仅需要 question 和 context ，负面样本可以自动生成。评估如果性能不佳，可以引入微调方案。

具体参见 [c-mteb](c-mteb/c-mteb.md)，但是这种方法只能评估 Embedding 模型，如果你的检索是混合检索（比如混合了 BM25 + Embedding + Rerank），那么就无法通过这种方法评估检索效果。

## 4. 运行 + 评估

In [ ]:
# Provider 为 rag.py，会继承当前 shell 的环境变量
# 执行 eval
!OPENAI_API_KEY="xxxx" langeval -v run cmrc-eval-zh.yaml --sample

# 执行批量的 eval
# export OPENAI_API_KEY = ""
# langeval run cmrc-eval-zh.yaml

# 实现了 生成正确率和检索召回率的计算，具体指标请参考文档。